In [ ]:
# clone repo and install dependences
!pip install --upgrade --no-cache-dir gdown
from IPython.display import clear_output
!git clone https://github.com/siidev/StyleMapGAN.git
%cd StyleMapGAN/
!pip install ninja
!pip install flask-ngrok
!pip install scikit-image==0.16.2
clear_output()
print('Done!')

Done!


In [ ]:
!gdown --id 1D90rlHutp7bklhk9BkF2RF_ZdNVTlO9s
!unzip celeba_hq.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1D90rlHutp7bklhk9BkF2RF_ZdNVTlO9s
To: /content/StyleMapGAN/celeba_hq.zip
100% 1.39G/1.39G [00:08<00:00, 162MB/s]
Archive:  celeba_hq.zip
   creating: lmdb-data/celeba_hq/
   creating: lmdb-data/celeba_hq/test/
  inflating: lmdb-data/celeba_hq/test/data.mdb  
  inflating: lmdb-data/celeba_hq/test/lock.mdb  
   creating: lmdb-data/celeba_hq/val/
  inflating: lmdb-data/celeba_hq/val/data.mdb  
  inflating: lmdb-data/celeba_hq/val/lock.mdb  
   creating: lmdb-data/celeba_hq/train/
  inflating: lmdb-data/celeba_hq/train/data.mdb  
  inflating: lmdb-data/celeba_hq/train/lock.mdb  


Read https://github.com/naver-ai/StyleMapGAN/blob/main/metrics/README.md

In [ ]:
# prepare fid
import gdown
import os

url = 'https://drive.google.com/uc?id=1pCr4lNCON7IZcNVdskIDXhFJ3jYuge1w' # inception checkpoint
output = 'metrics/pt_inception-2015-12-05-6726825d.pth'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=1dMHaNkgIy_-cq70f6nqoQsfG9-aOTpSC' # random generated samples as pickle files
output = 'fid_stats.zip'
gdown.download(url, output, quiet=False)
!unzip fid_stats.zip

Downloading...
From: https://drive.google.com/uc?id=1pCr4lNCON7IZcNVdskIDXhFJ3jYuge1w
To: /content/StyleMapGAN/metrics/pt_inception-2015-12-05-6726825d.pth
100%|██████████| 95.6M/95.6M [00:00<00:00, 242MB/s]

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1dMHaNkgIy_-cq70f6nqoQsfG9-aOTpSC 



Access denied with the following error:
unzip:  cannot find or open fid_stats.zip, fid_stats.zip.zip or fid_stats.zip.ZIP.


In [ ]:
# download celeba_hq checkpoint
checkpoint = '700000.pt'
!gdown --id 1D3XOuEo5nd4amyEZmlMf2IlBdu6MQCZJ

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1D3XOuEo5nd4amyEZmlMf2IlBdu6MQCZJ
To: /content/StyleMapGAN/700000.pt
100% 1.05G/1.05G [00:08<00:00, 129MB/s]


In [ ]:
# MSE & LPIPS
!python generate.py --ckpt {checkpoint} --mixing_type reconstruction --test_lmdb lmdb-data/celeba_hq/test
!python -m metrics.reconstruction --image_folder_path expr/reconstruction/celeba_hq

Namespace(batch=16, batch_per_gpu=8, channel_multiplier=2, ckpt='700000.pt', d_reg_every=16, dataset='celeba_hq', iter=1400000, lambda_adv_loss=1, lambda_d_loss=1, lambda_indomainGAN_D_loss=1, lambda_indomainGAN_E_loss=1, lambda_perceptual_loss=1, lambda_w_rec_loss=1, lambda_x_rec_loss=1, latent_channel_size=64, latent_spatial_size=8, local_editing_part=None, lr=0.002, lr_mul=0.01, mapping_layer_num=8, mixing_type='reconstruction', ngpus=1, normalize_mode='LayerNorm', num_workers=10, r1=10, save_image_dir='expr', save_network_interval=2000, size=256, small_generator=True, start_iter=694000, test_lmdb='lmdb-data/celeba_hq/test', train_data='lmdb-data/celeba_hq/train', val_data='lmdb-data/celeba_hq/val')
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that

In [ ]:
# FID
!python -m metrics.fid --ckpt {checkpoint} --comparative_fid_pkl metrics/fid_stats/celeba_hq_stats_256_29000.pkl --dataset celeba_hq

/usr/local/lib/python3.7/dist-packages/torchvision/models/inception.py:83: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  ' due to scipy/scipy#11299), please set init_weights=True.', FutureWarning)
  0% 0/1813 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:3635: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode)
100% 1813/1813 [1:09:55<00:00,  2.31s/it]
extracted 29000 features
fid: 6.778459739870101


In [ ]:
!gdown --id 1n_7_fI0bzyxbo3fo8jtl7bk7qJ8Vij8P # download data for local editing metric calculation
!unzip local-editing-metric-celeba-hq.zip

/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1n_7_fI0bzyxbo3fo8jtl7bk7qJ8Vij8P
To: /content/StyleMapGAN/local-editing-metric-celeba-hq.zip
100% 73.7M/73.7M [00:00<00:00, 92.6MB/s]
Archive:  local-editing-metric-celeba-hq.zip
   creating: data/
   creating: data/celeba_hq/
   creating: data/celeba_hq/LMDB_test/
  inflating: data/celeba_hq/LMDB_test/data.mdb  
  inflating: data/celeba_hq/LMDB_test/lock.mdb  
   creating: data/celeba_hq/local_editing/
  inflating: data/celeba_hq/local_editing/CelebA-HQ-to-CelebA-mapping.txt  
  inflating: data/celeba_hq/local_editing/celeba_hq_test_GT_sorted_pair.pkl  
   creating: data/celeba_hq/local_editing/GT_labels/
  inflating: data/celeba_hq/local_editing/GT_labels/10110.png  
  inflating: data/celeba_hq/local_editing/GT_

In [ ]:
# hair, background, eye, eyebrow, lip, neck, cloth, skin, ear
!python generate.py --ckpt {checkpoint} --mixing_type local_editing --test_lmdb lmdb-data/celeba_hq/test --local_editing_part nose
!python generate.py --ckpt {checkpoint} --mixing_type local_editing --test_lmdb lmdb-data/celeba_hq/test --local_editing_part hair
!python generate.py --ckpt {checkpoint} --mixing_type local_editing --test_lmdb lmdb-data/celeba_hq/test --local_editing_part background
!python generate.py --ckpt {checkpoint} --mixing_type local_editing --test_lmdb lmdb-data/celeba_hq/test --local_editing_part eye
!python generate.py --ckpt {checkpoint} --mixing_type local_editing --test_lmdb lmdb-data/celeba_hq/test --local_editing_part eyebrow
!python generate.py --ckpt {checkpoint} --mixing_type local_editing --test_lmdb lmdb-data/celeba_hq/test --local_editing_part lip
!python generate.py --ckpt {checkpoint} --mixing_type local_editing --test_lmdb lmdb-data/celeba_hq/test --local_editing_part cloth
!python generate.py --ckpt {checkpoint} --mixing_type local_editing --test_lmdb lmdb-data/celeba_hq/test --local_editing_part skin
!python generate.py --ckpt {checkpoint} --mixing_type local_editing --test_lmdb lmdb-data/celeba_hq/test --local_editing_part ear

Namespace(batch=16, batch_per_gpu=8, channel_multiplier=2, ckpt='700000.pt', d_reg_every=16, dataset='celeba_hq', iter=1400000, lambda_adv_loss=1, lambda_d_loss=1, lambda_indomainGAN_D_loss=1, lambda_indomainGAN_E_loss=1, lambda_perceptual_loss=1, lambda_w_rec_loss=1, lambda_x_rec_loss=1, latent_channel_size=64, latent_spatial_size=8, local_editing_part='nose', lr=0.002, lr_mul=0.01, mapping_layer_num=8, mixing_type='local_editing', ngpus=1, normalize_mode='LayerNorm', num_workers=10, r1=10, save_image_dir='expr', save_network_interval=2000, size=256, small_generator=True, start_iter=694000, test_lmdb='lmdb-data/celeba_hq/test', train_data='lmdb-data/celeba_hq/train', val_data='lmdb-data/celeba_hq/val')
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware tha

In [ ]:
!python -m metrics.local_editing --data_dir expr/local_editing/celeba_hq

parts ['lip', 'hair', 'cloth', 'ear', 'eyebrow', 'eye', 'nose', 'skin', 'background']
lip
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
100% 63/63 [00:07<00:00,  8.24it/s]
0.073, 0.031
hair
100% 63/63 [00:04<00:00, 15.56it/s]
0.030, 0.175
cloth
100% 63/63 [00:03<00:00, 15.94it/s]
0.049, 0.047
ear
100% 63/63 [00:03<00:00, 15.84it/s]
0.090, 0.047
eyebrow
100% 63/63 [00:03<00:00, 16.04it/s]
0.096, 0.030
eye
100% 63/63 [00:03<00:00, 15.90it/s]
0.108, 0.031
nose
100% 63/63 [00:04<00:00, 15.59it/s]
0.040, 0.034
skin
100% 63/63 [00:04<00:00, 13.87it/s]
0.027, 0.108
background
100% 63/63 [0

In [ ]:
# !mv expr expr_light
!zip -r expr_light.zip expr_light

Streaming output truncated to the last 5000 lines.
  adding: expr_light/local_editing/celeba_hq/nose/reference_reconstruction/166.png (deflated 0%)
  adding: expr_light/local_editing/celeba_hq/nose/reference_reconstruction/227.png (deflated 0%)
  adding: expr_light/local_editing/celeba_hq/nose/reference_reconstruction/171.png (deflated 0%)
  adding: expr_light/local_editing/celeba_hq/nose/reference_reconstruction/145.png (deflated 0%)
  adding: expr_light/local_editing/celeba_hq/nose/reference_reconstruction/132.png (deflated 0%)
  adding: expr_light/local_editing/celeba_hq/nose/reference_reconstruction/73.png (deflated 0%)
  adding: expr_light/local_editing/celeba_hq/nose/reference_reconstruction/9.png (deflated 0%)
  adding: expr_light/local_editing/celeba_hq/nose/reference_reconstruction/271.png (deflated 0%)
  adding: expr_light/local_editing/celeba_hq/nose/reference_reconstruction/174.png (deflated 0%)
  adding: expr_light/local_editing/celeba_hq/nose/reference_reconstruction/21.p

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!cp expr_light.zip /content/drive/MyDrive/stylemapgan/benchmark/